# Credit Risk Resampling Techniques

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [30]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [31]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [32]:
# Create our features
X = df.drop('loan_status', axis=1)

# Create our target
y = df.loan_status

In [33]:
X=pd.get_dummies(X)

In [34]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [35]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [37]:
X_test

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
1033,24000.0,0.0819,488.82,55080.0,26.74,0.0,0.0,9.0,0.0,17162.0,...,1,1,0,1,0,1,1,0,1,1
1219,14400.0,0.2000,535.16,62691.2,24.58,0.0,2.0,38.0,0.0,12190.0,...,1,1,0,1,0,1,1,0,1,1
62328,9000.0,0.0756,280.21,110000.0,16.57,0.0,1.0,20.0,1.0,13348.0,...,0,1,0,1,1,0,1,0,1,1
5966,25000.0,0.1171,826.90,100000.0,23.69,0.0,0.0,8.0,0.0,53394.0,...,1,1,1,0,0,1,1,0,1,1
24178,35000.0,0.1502,833.02,125000.0,27.87,0.0,0.0,15.0,0.0,42121.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18266,5000.0,0.1640,176.78,60000.0,18.84,0.0,0.0,13.0,0.0,6340.0,...,0,1,0,1,1,0,1,0,1,1
9773,26000.0,0.0756,809.48,60000.0,15.08,0.0,0.0,7.0,0.0,23857.0,...,1,1,0,1,0,1,1,0,1,1
18052,10000.0,0.1719,357.48,55000.0,26.69,1.0,0.0,5.0,0.0,6862.0,...,0,1,0,1,1,0,1,0,1,1
5729,15000.0,0.0756,467.01,65000.0,21.97,0.0,0.0,10.0,1.0,9961.0,...,1,1,0,1,0,1,0,1,1,1


In [38]:
X_train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
45047,10000.0,0.0819,314.25,106000.0,10.51,0.0,0.0,9.0,0.0,8248.0,...,0,1,0,1,1,0,1,0,1,1
3221,35000.0,0.1240,785.65,115000.0,25.30,0.0,0.0,13.0,0.0,4.0,...,1,1,1,0,0,1,0,1,1,1
63821,18000.0,0.1797,456.79,58000.0,23.53,0.0,0.0,19.0,0.0,22370.0,...,0,1,1,0,0,1,1,0,1,1
60268,40000.0,0.1614,975.71,48000.0,30.85,1.0,0.0,9.0,0.0,6306.0,...,0,1,0,1,0,1,0,1,1,1
26061,3150.0,0.1131,103.60,84000.0,13.23,0.0,1.0,14.0,1.0,6240.0,...,0,1,1,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,11000.0,0.1072,237.64,85000.0,21.08,0.0,1.0,25.0,0.0,19146.0,...,0,1,0,1,1,0,0,1,1,1
6265,30000.0,0.2000,1114.91,110000.0,14.12,3.0,1.0,7.0,0.0,718.0,...,1,1,0,1,0,1,1,0,1,1
54886,40000.0,0.1033,856.40,85000.0,19.02,0.0,0.0,10.0,0.0,13913.0,...,0,1,0,1,0,1,0,1,1,1
860,20000.0,0.1474,690.77,101447.0,16.19,0.0,1.0,52.0,0.0,19955.0,...,1,1,0,1,0,1,1,0,1,1


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [39]:
Counter(y_train)

Counter({'low_risk': 51369, 'high_risk': 243})

In [40]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51369, 'high_risk': 51369})

### Counter

In [ ]:
Counter

In [9]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

Counter({'loan_status': 1})

In [10]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [41]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='lbfgs', random_state=1)
log_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [11]:
# Calculated the balanced accuracy score
# YO

0.646602844334948

In [42]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

predictions = log_model.predict(X_test)

balanced_accuracy_score(y_test, predictions)

0.6297447180326836

### Display the confusion matrix
# YOUR CODE HERE

In [44]:
#Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[   59,    45],
       [ 5264, 11837]])

In [13]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.74      0.55      0.02      0.64      0.42       101
   low_risk       1.00      0.55      0.74      0.71      0.64      0.40     17104

avg / total       0.99      0.55      0.74      0.71      0.64      0.40     17205



In [45]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.57      0.69      0.02      0.63      0.39       104
   low_risk       1.00      0.69      0.57      0.82      0.63      0.40     17101

avg / total       0.99      0.69      0.57      0.81      0.63      0.40     17205



### SMOTE Oversampling

In [14]:
# Resample the training data with SMOTE
# YOUR CODE HERE

Counter({'loan_status': 1})

In [46]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1, sampling_strategy=1.0)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51369, 'high_risk': 51369})

In [15]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [47]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
s_model = LogisticRegression(solver='lbfgs', random_state=1)
s_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [16]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.662394124702461

In [52]:
# Calculated the balanced accuracy score
predictions = s_model.predict(X_test)

balanced_accuracy_score(y_test, predictions)

0.6477134715468731

In [17]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   64,    37],
       [ 5283, 11821]])

In [50]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   58,    46],
       [ 4485, 12616]])

In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.69      0.02      0.66      0.44       101
   low_risk       1.00      0.69      0.63      0.82      0.66      0.44     17104

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



In [51]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.56      0.74      0.02      0.64      0.40       104
   low_risk       1.00      0.74      0.56      0.85      0.64      0.42     17101

avg / total       0.99      0.74      0.56      0.84      0.64      0.42     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [19]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

Counter({'loan_status': 1})

In [72]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids

clu_cent = ClusterCentroids(random_state=1)
X_resampled, y_resampled = clu_cent.fit_resample(X_test, y_test)
Counter(y_resampled)

Counter({'high_risk': 104, 'low_risk': 104})

In [20]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [73]:
# Train the Logistic Regression model using the resampled data

c_model = LogisticRegression(solver='lbfgs', random_state=1)
c_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.5442166848817717

In [74]:
# Calculated the balanced accuracy score
from sklearn.model_selection import train_test_split

predictions = c_model.predict(x_test)
balanced_accuacy-score(y_test,predictions)

NameError: name 'x_test' is not defined

In [66]:
# Display the confusion matrix
# YOUR CODE HERE

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.42      0.01      0.53      0.29       101
   low_risk       1.00      0.42      0.67      0.59      0.53      0.27     17104

avg / total       0.99      0.42      0.67      0.58      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

Counter({'loan_status': 1})

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete




In [25]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(random_state=1)

In [26]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.6400726134353378

In [27]:
# Display the confusion matrix
# YOUR CODE HERE

array([[  71,   30],
       [7232, 9872]])

In [28]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.58      0.02      0.64      0.41       101
   low_risk       1.00      0.58      0.70      0.73      0.64      0.40     17104

avg / total       0.99      0.58      0.70      0.73      0.64      0.40     17205

